# Intro
This notebook is designed to updates cameras status (updating last ping and last image) 

It works as it is (just run blocks from the setup section)
It is also possible to run the entire notebook at once: To do so, in the menu, click ‘Run’ then ‘Run All Cells’.

In [ ]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from api import get_token, get_camera_token
from pyroclient import Client
import glob
from PIL import Image
import itertools
import io
from utils import dl_seqs_from_url

In [ ]:
API_URL = "http://api:5050"
load_dotenv("../.env")
SUPERADMIN_LOGIN = os.environ.get("SUPERADMIN_LOGIN")
SUPERADMIN_PWD = os.environ.get("SUPERADMIN_PWD")

# Get access token
admin_access_token = get_token(API_URL, SUPERADMIN_LOGIN, SUPERADMIN_PWD)

api_client = Client(admin_access_token, API_URL)

# get cameras -> in order to get cam name from cam id
cameras = api_client.fetch_cameras().json()

In [ ]:
# download img folder 
BASE_DIRECTORY = "../data"

SAMPLE_PATH = "last_image_cameras"
url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/last_image_cameras.zip"

if not os.path.isdir(os.path.join(BASE_DIRECTORY, SAMPLE_PATH)):
    print("Images not found, dowloading ...")
    dl_seqs_from_url(url, os.path.join(BASE_DIRECTORY, SAMPLE_PATH))
else:
    print(f"SAMPLE_PATH {SAMPLE_PATH} directory exists, assume alerts inside, no download")


In [ ]:
cam_imgs = glob.glob(os.path.join(BASE_DIRECTORY, SAMPLE_PATH,"*"))
images_cycle = itertools.cycle(cam_imgs)

# updates hearbeat and last image for cameras
for cam in cameras:   
    camera_token = get_camera_token(API_URL, cam["id"], admin_access_token)
    camera_client  = Client(camera_token, API_URL)

    # update camera heartbeat/lastping
    camera_client.heartbeat()

    # update camera last image
    img_file = next(images_cycle)
    
    stream = io.BytesIO()
    im = Image.open(img_file)
    im.save(stream, format="JPEG", quality=80)
    
    response = camera_client.update_last_image(stream.getvalue())

print("hearbeat and last image updates with success")